<a href="https://colab.research.google.com/github/tgawade/genai/blob/main/FpCustReview9mar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 183.2 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Non-user install by explicit request
  Created build tracker: /tmp/pip-build-tracker-zje770di
  Entered build tracker: /tmp/pip-build-tracker-zje770di
  Created temporary directory: /tmp/pip-install-gu1blb7j
  Created temporary directory: /tmp/pip-ephem-wheel-cache-ur4tru_b
  1 location(s) to search for versions of scikit-build-core:
  * https://pypi.org/simple/scikit-build-core/
  Fetching project page and analyzing links: https://pypi.org/simple/scikit-build-core/
  Getting page https://pypi.org/simple/scikit-build-core/
  Found index url https://pypi.org/simple/
  Looking up "https://pypi.org/simple/scikit-build-core/" in the cache
  Request header has "max_age" as 0, cache bypasse

In [53]:
import torch

In [54]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [55]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

  Using cached llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.2.28
    Uninstalling llama_cpp_python-0.2.28:
      Successfully uninstalled llama_cpp_python-0.2.28


In [56]:
!pip install evaluate
!pip install bert-score

In [16]:
!pip install --upgrade numpy
!pip install --upgrade scikit-learn

In [10]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.3 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
pytensor 2.27.1 requires numpy<2,>=1.17.0, but you have numpy 2.2.3 which is incompatible.


In [20]:
!pip install --upgrade numpy
!pip install --upgrade scikit-learn
!pip install --upgrade scipy


In [10]:
!pip install --upgrade numpy==1.23.5

  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatib

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

In [3]:
# from google.colab import drive
import locale

In [4]:
import evaluate

In [5]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [6]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [7]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [9]:
system_message = """Classify the sentiment of movthe customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative'"""

In [10]:
user_input="""The customer made a purchase order with an agreement for two-day shipping but didn't receive the product due to high
demand and delayed shipment. The company informed the customer about the delay, suggesting to keep the order or cancel it. The customer
 wanted to cancel it and followed the company's policy by requesting the cancellation via email. After the order was canceled, the company
 verified the cancellation and issued a refund. However, the customer noted a discrepancy in refund claiming they were () charged for the
 canceled product. Though they disputed the issue, it was initially ruled in the company's favor. Capital One removed the mention of the purchase
 from the bill, but it reappeared later. The customer is now in a dispute over the recharged amount and is waiting for the final ruling on the case."""

In [31]:
zero_shot_prompt_template="""<s>[INST] {system_message} [/INST]```{user_message}```</s>"""

In [32]:
def generate_prompt(system_message,user_input):
    prompt=zero_shot_template.format(system_message=system_message,user_message=user_input)
    return prompt

In [46]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message,input_text)

    # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
            prompt=prompt,
            max_tokens=1200,
            temperature=0,
            top_p=0.95,
            repeat_penalty=1.2,
            top_k=50,
            stop=["/s"], # Dynamic stopping when such token is detected.
            echo=False # do not return the prompt
        )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
file_path = '/content/drive/MyDrive/Complaints_classification_2.csv'
data = pd.read_csv(file_path)

In [109]:
data.head()

,product,narrative,summary
0,credit_card,purchase order day shipping amount receive pro...,The customer made a purchase order with an agr...
1,credit_card,forwarded message date tue subject please inve...,The sender of the email believes they have bee...
2,retail_banking,forwarded message cc sent friday pdt subject f...,The sender of the email alleges that Wells Far...
3,credit_reporting,payment history missing credit report speciali...,The credit report from Specialized Loan Servic...
4,credit_reporting,payment history missing credit report made mis...,The text concerns a person who found an unauth...


In [110]:
# Randomly select 30 rows
new_data = data.sample(n=30, random_state=40)

In [47]:
# example - new_data['mistral_response'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =      31.99 ms /    55 runs   (    0.58 ms per token,  1719.34 tokens per second)
llama_print_timings: prompt eval time =     332.36 ms /   215 tokens (    1.55 ms per token,   646.90 tokens per second)
llama_print_timings:        eval time =    1626.95 ms /    54 runs   (   30.13 ms per token,    33.19 tokens per second)
llama_print_timings:       total time =    2120.21 ms /   269 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =      51.81 ms /    69 runs   (    0.75 ms per token,  1331.79 tokens per second)
llama_print_timings: prompt eval time =     510.33 ms /   505 tokens (    1.01 ms per token,   989.56 tokens per second)
llama_print_timings:        eval time =    2044.65 ms /    68 runs   (   30.07 ms per token,    33.26 tokens per second)
llama_print_timings:       to

In [21]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

In [48]:
new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:extract_category(x) )

In [49]:
new_data.head()

,product,narrative,summary,mistral_response,mistral_response_cleaned
167,retail_banking,fraudulent charge totaling made capital one ch...,A fraudulent charge was made on the individual...,negotative\n\nThis customer review expresses f...,
169,credit_reporting,block except otherwise provided section consum...,The text outlines various stipulations regardi...,I'm unable to classify the sentiment of the c...,
461,credit_card,usaa master plan collect cancellation debt usa...,The input appears to be a complaint about USAA...,negotiating with USAA to resolve multiple disp...,
253,credit_reporting,block except otherwise provided section consum...,The text pertains to the stipulations and oper...,I'm unable to classify the sentiment of the c...,
42,credit_reporting,open account acct opened balance account acct ...,The input is about various accounts being open...,Neutral (this text does not contain any senti...,


In [50]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [51]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['product'], new_data['mistral_response'],average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.0


**F1 score explanation: Since theere are no categories f1 and f2 scores are 0 for zero shot prompting **

In [111]:
import json
review_1 = data[data['product'] == 'credit_card']
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

In [112]:
gold_examples_df = data.drop(index=examples_df.index)
gold_examples_df.head()

,product,narrative,summary
0,credit_card,purchase order day shipping amount receive pro...,The customer made a purchase order with an agr...
1,credit_card,forwarded message date tue subject please inve...,The sender of the email believes they have bee...
2,retail_banking,forwarded message cc sent friday pdt subject f...,The sender of the email alleges that Wells Far...
3,credit_reporting,payment history missing credit report speciali...,The credit report from Specialized Loan Servic...
4,credit_reporting,payment history missing credit report made mis...,The text concerns a person who found an unauth...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('product').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('narrative').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('summary').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['narrative'].value_counts()
    for x_label, grp in _df_3.groupby('product')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('product')
_ = plt.ylabel('narrative')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['summary'].value_counts()
    for x_label, grp in _df_4.groupby('narrative')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('narrative')
_ = plt.ylabel('summary')

In [113]:
# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

In [114]:
print(json.loads(examples_json)[0])

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}


In [195]:
system_message = """Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' """

In [196]:
user_input_example="The sender of the email believes they have been scammed by Comenity Bank, a credit card provider company for retailers like Children's Place, New York Forever, and Victoria Secret. They noticed that their original credit limit was lowered, leading to overage and late fees. Upon contacting Comenity Bank, they were told the credit limit could return to its original limit, but only once payment for the account's fees was settled. This experience has negatively affected the sender's credit score. The sender is seeking help due to the described corruption concerning the fluctuating credit limit and major fees tied to their account that the bank is not willing to correct."

In [1]:
assistant_message="Sentiment[positive,negative]"

In [2]:
first_turn_template = """<s>[INST]{system_message}```{user_message}```[/INST]{assistant_message}</s> """
examples_template = """<s>[INST]```{user_message}```[/INST]{assistant_message}</s>"""
prediction_template = """<s>[INST]```{user_message}```[/INST]"""

In [3]:
def create_few_shot_prompt(system_message, examples_df):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    columns_to_select = ['narrative', 'product']
    examples = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples)):
        user_input_example = example['narrative']
        assistant_output_example = example['product']

        if idx == 0:
            few_shot_prompt += first_turn_template.format(
                system_message=system_message,
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )
        else:
            few_shot_prompt += examples_template.format(
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )

    return few_shot_prompt

In [4]:
few_shot_prompt = create_few_shot_prompt(system_message, examples_df)

NameError: name 'system_message' is not defined

In [201]:
print(few_shot_prompt)

<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' ```called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee```[/INST]credit_card</s> <s>[INST]```dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute making purchase credit card usaa two separate transaction merchant purchased trip cancelled told apply refund apply refund however company give refund back filed dispute credit agency help dispute account follow dispute policy state service provided customer right receive full refund impact credit negative way increasing credit balance decreasing credit 

In [202]:
def generate_prompt(few_shot_prompt,new_review):
    prompt =  few_shot_prompt + prediction_template.format(user_message=new_review)
    print(prompt)
    return prompt

In [206]:
def generate_mistral_response_1(support_ticket_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(few_shot_prompt,support_ticket_text)

    # Define the Llama model along with its parameters for generating a response
    #response =  # Define the Llama model along with its parameters for generating a response
    response = lcpp_llm(
            prompt=prompt,
            max_tokens=1200,
            temperature=0,
            top_p=0.95,
            repeat_penalty=1.2,
            top_k=50,
            stop=["/s"], # Dynamic stopping when such token is detected.
            echo=False # do not return the prompt
        )


    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    print(response_text)
    return response_text

In [208]:
# Randomly select 50 rows from gold_examples
new_data = gold_examples_df.sample(n=50, random_state=40)

In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response_1(x))

<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' ```called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee```[/INST]credit_card</s> <s>[INST]```dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute making purchase credit card usaa two separate transaction merchant purchased trip cancelled told apply refund apply refund however company give refund back filed dispute credit agency help dispute account follow dispute policy state service provided customer right receive full refund impact credit negative way increasing credit balance decreasing credit 

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =     218.37 ms /   154 runs   (    1.42 ms per token,   705.23 tokens per second)
llama_print_timings: prompt eval time =    1007.38 ms /   658 tokens (    1.53 ms per token,   653.18 tokens per second)
llama_print_timings:        eval time =    5709.55 ms /   153 runs   (   37.32 ms per token,    26.80 tokens per second)
llama_print_timings:       total time =    8059.93 ms /   811 tokens
Llama.generate: prefix-match hit


debt\_collection, identity\_theft, fair\_housing\_discrimination. The sentiment in this text is negative as it expresses frustration and anger towards a financial institution for various issues related to identity theft, incorrect credit reporting, unfair business practices, and discrimination. The author has experienced multiple difficulties with the company, including fraudulent activity on their account, denied access to credit due to inaccurate information, and being charged for services they did not receive or need. They have had to go through lengthy processes to dispute errors and resolve issues, and feel that they have been mistreated by the company despite following all necessary procedures and regulations. The text also expresses a desire for accountability and justice for the unethical business practices observed.
<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the i


llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =     138.19 ms /   227 runs   (    0.61 ms per token,  1642.68 tokens per second)
llama_print_timings: prompt eval time =     586.08 ms /   433 tokens (    1.35 ms per token,   738.81 tokens per second)
llama_print_timings:        eval time =    8027.29 ms /   226 runs   (   35.52 ms per token,    28.15 tokens per second)
llama_print_timings:       total time =    9483.80 ms /   659 tokens
Llama.generate: prefix-match hit


debt_collection, credit\_reporting

The text appears to discuss a situation where someone's identity was stolen and fraudulent accounts were opened in their name. The person is trying to have the incorrect information removed from their credit report and blocked from being reported by the consumer reporting agency (CRD). They have provided proof of identity theft, such as a police report, and have requested that the CRD block the disputed information. However, they are having difficulty getting the CRD to comply with their request and have been told that the resellers of the fraudulent accounts may still be able to access and sell the information. The person is also concerned about the potential impact on their credit score and ability to obtain loans or credit in the future. They mention a provision in the law that requires the CRD to block the reporting of disputed information if it results from identity theft, but they are having trouble getting the CRD to follow this requirement. T

...

debt_collection, identity\_theft
<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' ```called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee```[/INST]credit_card</s> <s>[INST]```dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute making purchase credit card usaa two separate transaction merchant purchased trip cancelled told apply refund apply refund however company give refund back filed dispute credit agency help dispute account follow dispute policy state service provided customer right receive full refund impact credit negative way increasing 


llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =      72.68 ms /   118 runs   (    0.62 ms per token,  1623.60 tokens per second)
llama_print_timings: prompt eval time =     236.93 ms /   104 tokens (    2.28 ms per token,   438.95 tokens per second)
llama_print_timings:        eval time =    4149.50 ms /   117 runs   (   35.47 ms per token,    28.20 tokens per second)
llama_print_timings:       total time =    4837.87 ms /   221 tokens
Llama.generate: prefix-match hit


credit\_reporting

The sentiment of the customer is negative, as they are expressing frustration and dissatisfaction with their experience related to identity theft and issues with credit reports. They have had multiple accounts opened fraudulently in their name, resulting in incorrect information being listed on their credit report, which has negatively impacted them financially and hindered their ability to make purchases or obtain loans due to the delinquent status of these accounts. The customer is also experiencing difficulty resolving the issue with the companies involved and feels defrauded by the situation.
<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' ```called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging lat


llama_print_timings:        load time =    1041.46 ms
llama_print_timings:      sample time =      43.25 ms /    73 runs   (    0.59 ms per token,  1687.82 tokens per second)
llama_print_timings: prompt eval time =     597.67 ms /   422 tokens (    1.42 ms per token,   706.07 tokens per second)
llama_print_timings:        eval time =    2641.26 ms /    72 runs   (   36.68 ms per token,    27.26 tokens per second)
llama_print_timings:       total time =    3501.26 ms /   494 tokens
Llama.generate: prefix-match hit


 This text appears to be a legal document discussing the process for blocking and resolving identity theft issues related to credit reports. The sentiment is neutral as it does not expressly state whether the situation described is positive or negative, but rather outlines the steps that should be taken in case of identity theft and the consumer's rights under the law regarding this matter.
<s>[INST]Classify the sentiment of the customer presented in the input as 'positive' or 'negative'.
Customer reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative' ```called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee```[/INST]credit_card</s> <s>[INST]```dispute case card ending usaa credit card merchant name transaction date transaction amount disputed amount dispute case card ending usaa credit card merchant name transaction date trans

In [191]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: extract_category(x))

In [192]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 =  f1_score(new_data['mistral_response_cleaned'], new_data['mistral_response'], average='micro')

print(f'F1 Score: {f1}')

F1 Score: 0.1


In [193]:
new_data.head(15)

,product,narrative,summary,mistral_response,mistral_response_cleaned
74,credit_reporting,anyone us credit report another consumer repor...,The narrator is filing a complaint against ind...,"debt\_collection, identity\_theft, fair\_housi...",
174,credit_reporting,block except otherwise provided section consum...,The input refers to regulations for consumer r...,"debt_collection, credit\_reporting",debt_collection
85,credit_reporting,shocked reviewed credit report found late paym...,The user reviewed their credit report and was ...,credit\_reporting\n\nThe sentiment of the cust...,
372,credit_reporting,except otherwise provided section consumer rep...,This passage discusses the procedures and requ...,credit_reporting\n\nThe sentiment of the custo...,credit_reporting
296,credit_reporting,block except otherwise provided section consum...,The input text discusses the obligations of co...,"debt_collection, credit\_reporting",debt_collection
247,credit_reporting,block except otherwise provided section consum...,The document discusses rules regarding blockin...,"debt_collection, identity\_theft",debt_collection
386,credit_reporting,except otherwise provided section consumer rep...,This section mandates consumer reporting agenc...,credit_reporting\n\nThe sentiment of the custo...,credit_reporting
51,debt_collection,left apartment charging u usage credence colle...,The writer has left their apartment and has be...,housing\n\nThe sentiment of the customer's rev...,
107,debt_collection,calling friend family employer extended family...,The text is about someone trying to locate an ...,debt_collection,debt_collection
347,credit_reporting,blockexcept otherwise provided section consume...,The text outlines regulations for a consumer r...,"debt_collection, credit\_reporting",debt_collection


In [ ]:
system_message = ""